In [37]:
import pandas as pd

df = pd.read_csv("data/polling_data.csv")

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18095 entries, 0 to 18094
Data columns (total 52 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   poll_id                    18095 non-null  int64  
 1   pollster_id                18095 non-null  int64  
 2   pollster                   18095 non-null  object 
 3   sponsor_ids                8482 non-null   object 
 4   sponsors                   8482 non-null   object 
 5   display_name               18095 non-null  object 
 6   pollster_rating_id         18095 non-null  int64  
 7   pollster_rating_name       18095 non-null  object 
 8   numeric_grade              15901 non-null  float64
 9   pollscore                  15915 non-null  float64
 10  methodology                16995 non-null  object 
 11  transparency_score         14886 non-null  float64
 12  state                      10071 non-null  object 
 13  start_date                 18095 non-null  obj

In [41]:
# Copy original data for cleaning
battleground_states = ['Georgia', 'North Carolina', 'Pennsylvania', 'Michigan', 'Wisconsin', 'Nevada', 'Arizona']
mask = (
        (df["answer"].isin(["Trump", "Harris"])) &
        (df['state'].isin(battleground_states))
)
clean_data = df[mask].copy()
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3331 entries, 7 to 18060
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   state           3331 non-null   object 
 1   answer          3331 non-null   object 
 2   candidate_name  3331 non-null   object 
 3   pct             3331 non-null   float64
dtypes: float64(1), object(3)
memory usage: 130.1+ KB


In [40]:
# Extract only relevant states
clean_data["state"]

7             Arizona
8             Arizona
11            Arizona
12            Arizona
13            Georgia
             ...     
17953        Michigan
17955    Pennsylvania
17957       Wisconsin
17977       Wisconsin
18060         Arizona
Name: state, Length: 3331, dtype: object